## Introduction
This file is concerned with the first asssignment in the ST 443 group project. The task is to classify the observation to one of the eight vegetation classes based on the reflectance values for each pixel in the i-th wavelenght band, i $ \ \in \ \{1, 2, \ldots, 218\}$. We will start with T1.1, which is concerned with data visualization and understanding the distribution of the features, and the target.

## Data Preparation and Imports

### Import

In [ ]:
#Imports from the standard library
import sys

#Third-party imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    balanced_accuracy_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)
from sklearn.model_selection import cross_validate, StratifiedKFold


### Read the zipped csv file

In [ ]:
#Replace the path with the absolute path of your file
data1=pd.read_csv('data-1.csv.gz')

## Task 1.1: Data Visualization and summary statistics

Inspect the dataset

In [ ]:
#Shape of the dataframe
print(f"\n Number of Rows: {data1.shape[0]} \n Number of Columns: {data1.shape[1]}")

In [ ]:
#Datatypes of all the columns in the dataset
data1.dtypes

In [ ]:
#Information about the dataset
data1.info()

In [ ]:
#Look at the first 5 entries
data1.head()

Summary statistics

In [ ]:
#Summary statistics of the float columns
data1.select_dtypes(include = "float64").describe()

Missing Values

In [ ]:
#Missing values
missing_features = data1.select_dtypes(include = "float64").isna().sum().sum()
missing_target = data1['land_type'].isna().sum()
print(f"Total missing values in the feature space: {missing_features}")
print(f"Total missing target values: {missing_target}")

Duplicates

In [ ]:
#Duplicates check
data1[data1.duplicated() == True]

No duplicates in this dataset. We can check for invalid values. Reflectance values should be in $[0,1]$.

In [ ]:
bands = [col for col in data1.columns if col.startswith("B")]
print("Negative reflectance values:", (data1[bands] < 0).sum().sum())
print("Reflectance values > 1:", (data1[bands] > 1).sum().sum())

There are invalid values in the dataset, so we should clip them, such that the minimum value is 0, and the maximum value is 1.

In [ ]:
#Clip the extreme values
data1[bands] = data1[bands].clip(lower=0, upper=1)
print("After clipping:")
print("Min reflectance:", data1[bands].min().min())
print("Max reflectance:", data1[bands].max().max())

Function to check for outliers based on the Z-scores.

In [ ]:
def outlier_check(df, cols, z):
    z_score = np.abs((df[cols] - df[cols].mean()) / df[cols].std())
    outliers_df = df[np.any(z_score > z, axis = 1)]
    return outliers_df.shape[0]

We can check for values of 3 standard deviations away from mean to identify outliers.

In [ ]:
outliers_number = outlier_check(data1, bands, 3)
percentage_of_outliers = outliers_number/data1.shape[0] * 100
print(f"Percentage of outliers: {percentage_of_outliers}")

Very few outliers, and we were very severe in flagging them. Normally, we would not expect features to follow a normal distribution, so there would be less outliers. No reason to drop them. We should check for imbalance by looking at the distribution of the vegetation classes.

Distribution of vegetation classes

In [ ]:

# Check distribution of vegetation classes to indentify potential imbalance(may affect classification later)
class_counts = data1["land_type"].value_counts()
class_percent = data1["land_type"].value_counts(normalize=True) * 100

balance_df = pd.DataFrame({
    "Count": class_counts,
    "Percentage": class_percent.round(2)
}).reset_index().rename(columns={"index": "Land Type"})


In [ ]:
sns.countplot(y="land_type", data=data1)
plt.title("Distribution of Vegetation Classes")
plt.show()

We can observe that the alpine meadopw is the most frequent vegetation class, with around a quarter of the observations classified in this class. Also, valley floor and alpine tundra are also quite prevalent, and about 58% of all the observations are classified into one of this vegetation types.

In [ ]:
# Spatial distribution of land types (confirms figure 1 in the project description)
sample = data1.sample(10000, random_state=0)
plt.figure(figsize=(6,6))
sns.scatterplot(x="p_x", y="p_y", hue="land_type", data=sample, s=8, linewidth=0)
plt.title("Spatial Distribution of Land Types")
plt.axis("equal")
plt.show()

We should check if classes are separable in the spectral space.

In [ ]:
# Check if classes are separable in spectral space
mean_spectra = data1.groupby("land_type")[bands].mean().T
mean_spectra

# Lines that differ strongly → those classes are spectrally separable → classification should work well.
# Overlapping lines → those classes are spectrally similar → may need nonlinear models (e.g. GBDT / SVM).
# Smoothness across bands → confirms that adjacent bands are highly correlated. Motivates PCA or regularisation later.

In [ ]:
mean_spectra.plot(figsize=(10,5))
plt.xlabel("Spectral band index (~420–2450 nm)")
plt.ylabel("Mean surface reflectance")
plt.title("Mean Spectral Signature by Land Type")
plt.legend(bbox_to_anchor=(1.05,1))
plt.show()

Note that for Lines that differ strongly it means those classes are spectrally separable, so classification should work well. For the Overlapping lines the classes are spectrally similar, so may need nonlinear models (e.g. GBDT / SVM).
There is smoothness across bands, which confirms that adjacent bands are highly correlated. Motivates PCA or regularisation later.

Inspect the correlations across the features

In [ ]:
#Correlation for the first 50 bands
corr50 = data1[bands[:50]].corr()
corr50.head()

In [ ]:
#Correlation heatmap for the first 50 bands
sns.heatmap(corr50, cmap="coolwarm", center=0)
plt.title("Correlation among first 50 bands")
plt.show()

In [ ]:
#Correlation among all the bands
corrfull = data1[bands].corr()
corrfull

In [ ]:

# Correlation heatmap for all bands
sns.heatmap(corrfull, cmap="coolwarm", center=0)
plt.title("Correlation among all bands")
plt.show()


In [ ]:
plt.scatter(data1["Band_1"], data1["Band_2"])
plt.xlabel("Band1")
plt.ylabel("Band2")
plt.title("Scatterplot of Band 1 and Band 2")
plt.show()

In [ ]:
plt.scatter((data1["Band_1"] - data1["Band_1"].mean())/data1["Band_1"].std(), (data1["Band_214"] - data1["Band_214"].mean())/data1["Band_214"].std())
plt.xlabel("Band 1")
plt.ylabel("Band 214")
plt.title("Scatterplot of Z-Scored Band 1 and Band 214")
plt.show()

In [ ]:
plt.scatter((data1["Band_217"] - data1["Band_217"].mean())/data1["Band_217"].std(), (data1["Band_214"] - data1["Band_214"].mean())/data1["Band_214"].std())
plt.xlabel("Band 217")
plt.ylabel("Band 214")
plt.title("Scatterplot of Z Scored Band 217 and Band 214")
plt.show()

We can observe that the correlation is larger among neighboring features. For example the average correlation is much larger when only the first 50 bands are considered than when all the bands are considered. Indeed, we can observe that correlation even becomes negative for some distant bands. We have defined distant bands in a very rough way, as the difference between distance labels.

We should also look at the conditional distributions of the bands to better understand the shape.

In [ ]:
plt.hist(data1[data1["land_type"] == "alpine meadow"]["Band_1"], bins = 100)
plt.show()

In [ ]:
plt.hist(data1["Band_1"], bins = 100)
plt.show()

In [ ]:
plt.hist(data1[data1["land_type"] == "snow / ice"]["Band_1"], bins = 100)
plt.show()

We can observe heterogeneous conditional distributions. In particular, Band 1 values are larger for the land type snow/ice, and many of the values achieve the maximumm threshold.

Before fitting the models, we will do two more tasks: Clustering and PCA. Clustering will helps us visualize the features more, and PCA will be able to help us for dimensionality reduction. Due to the large number of features, it might be helpful to do Clustering on the first principal components. We will apply PCA to the training set, and then clustering. PCA will help evaluate whether we can reduce dimensionality, while clustering will aid us in visualizing the features.

Find the bands

In [ ]:
X_bands = data1[bands]

Perform PCA using 4 components

In [ ]:
X_pca = StandardScaler().fit_transform(X_bands)
pca  = PCA(n_components=4).fit(X_bands)
print(f"\n Percentage of Variance explained by the first four principal components: {np.round(sum(pca.explained_variance_ratio_) * 100, 3)} %")

Percentage of explained Variance by each component

In [ ]:
principal_components = pd.DataFrame({"Component": [1, 2, 3, 4], "Percentage of Explained Variance": np.round(pca.explained_variance_ratio_ * 100, 3)}).set_index("Component")
principal_components

PCA singular values

In [ ]:
singular_values = pd.DataFrame({"Component": [1, 2, 3, 4], "Singular Value": pca.singular_values_}).set_index("Component")
singular_values

PCA eigenvectors

In [ ]:
eigenvectors = pd.DataFrame(pca.components_)
eigenvectors

We can observe that only the first three principal components capture 99% of the variance in the feature space, so we can retain only the frist 3 principal components. This suggests that dimensionality reduction is achievable on this dataset.

In [ ]:
Z = PCA(n_components=3).fit_transform(X_pca)

We will now perform clustering on the reduced dataset using K-Means. It is better to apply K-Means on the PCA transformed dataset for stability. For visualization purposes, we will perform clustering on the first two principal components only.


In [ ]:
pc1 = Z[:, 0]
pc2 = Z[:, 1]
kmeans = KMeans(n_clusters=3)
cluster = kmeans.fit_predict(Z[:, 0:2])

In [ ]:
scatter = plt.scatter(pc1, pc2, c = cluster)
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1],
            marker="o", s=200, edgecolor="k")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("K-Means Clusters in PCA Space")
plt.legend(*scatter.legend_elements(), title="Cluster")
plt.show()


The K-Means does not aid us too much in visualizing the features when we use the first 2 PCs. Clusters seem arbitrarily separated, and, indeed, the scatteplot does not suggest clustering in the PCA space. As such, we will only use PCA for dimensionality reduction and model fitting.

## Task 1.2 - Model Development

In this part of the project, we will try to predict the Vegetation type based on the reflectance values of the pixels. This is a MultiClass classification problem, and we will use several types of classifiers for this task. To encode the variables from categorical to numerical, we will use Scikit Learn's Label Encoder. The methodology will be the following: we will fit the models and report the cross-validation errors on the training set to adjust the hyperparamters. After finding the best model, we will fit it to the whole training data, and report its performance on the test set. The test set will be used only to report the performance.

### Define functions to evaluate performances

Create function to report cross-validation performance - This will help us tweak the hyperparameters in case it's necessary.

In [ ]:
def evaluate_cv(estimator, X, y, scoring_list, model_name = "model"):
    
    #Define StratifiedKFold splitter. This makes sure than no split is more imbalanced than the original data
    cv_splitter = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

    #Calculate the CV score for each metric
    cv_results = cross_validate(
        estimator, 
        X, 
        y, 
        scoring = scoring_list, 
        cv = cv_splitter, 
        return_train_score=False
    )

    #Create a dataframe with all the performance metrics using cv
    row = {"model": model_name}
    for k, v in cv_results.items():
        if k.startswith("test_"):
            metric = k.replace("test_", "")
            row[f"{metric}"] = np.mean(v)
        
    return pd.DataFrame([row]).set_index("model")

Function to evaluate the performance on the test set, and return the predictions

In [ ]:
def predict_evaluate(estimator, X_tr: pd.DataFrame, X_ts: pd.DataFrame, y_tr: pd.DataFrame, y_ts, model_name: str):

    #Fit the estimator on the training data
    classifier = estimator.fit(X_tr, y_tr)

    #Find the predictions and probabilities
    predictions = classifier.predict(X_ts)
    predictions_prob = classifier.predict_proba(X_ts)

    #Calculate the error metrics using predictions
    acc = accuracy_score(y_ts, predictions)
    balanced_acc = balanced_accuracy_score(y_ts, predictions)
    f1 = f1_score(y_ts, predictions, average="macro")

    #Calculate auroc using the probabilities
    auc = roc_auc_score(y_ts, predictions_prob, average = "macro", multi_class="ovr")

    return pd.DataFrame({"model": [model_name], "Accuracy": [acc], "Bal. Acc": [balanced_acc], "AUC": [auc], "F1": [f1]}), predictions


Generalization of the previous function to test the performance of multiple models

In [ ]:
def multiple_predict_evaluate(models:list[tuple], X_tr: pd.DataFrame, X_ts: pd.DataFrame, y_tr, y_ts):

    #Create a list of dataframes to hold the performances of each model
    metrics_list = []

    #Create a list of dataframes to hold the predictions from each model
    predictions_list = []

    #Iterate through each model
    for model, estimator in models:
        
        #report the performance and predicitions of the model
        metric, preds = predict_evaluate(estimator, X_tr, X_ts, y_tr, y_ts, model)

        #Append the performance to the metrics list
        metrics_list.append(metric)

        #Create dataframe to hold the predictions
        df_model = pd.DataFrame({f"{model}": preds})

        #Append the dataframe to the  predictions_list
        predictions_list.append(df_model)

    #Return the concatenated dataframes
    return pd.concat(metrics_list, ignore_index = True), pd.concat(predictions_list, axis = 1)

Function to plot the confusion matrix

In [ ]:
def plot_confusion_matrix(y_true, y_pred, class_names, normalize, model_name):

    #Create figure
    fig = plt.figure(figsize=(12, 12), dpi=100)  # FORCE a large, readable figure
    ax = fig.add_subplot(111)

    #Create Confusion Matrix display
    disp = ConfusionMatrixDisplay.from_predictions(
        y_true,
        y_pred,
        display_labels=class_names,
        normalize=normalize,
        cmap="Blues",
        colorbar=True,
        ax=ax
    )
    
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right")
    plt.title(f"Confusion Matrix - {model_name}", fontsize=20)
    plt.tight_layout()
    plt.show()

### Preparations of the features and the target

Find the features and the target

In [ ]:
features = data1.drop(columns=["land_type","rgb_hex", "overlay_hex"])
target  = data1["land_type"]

Split into a training set and a test set

In [ ]:
# General train/test split, for models\svm, knn
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    target, 
    test_size=0.2, 
    random_state=42, 
    shuffle=True,
    stratify=target #This ensures the train and test set follow the same distribution as the original dataset
)

In [ ]:
data1_1 = X_train.sample(30000, random_state=42)
y1_1 = y_train.loc[data1_1.index]

X_train_1 = data1_1
y_train_1 = y1_1

In [ ]:
print(f"\n Number of observations in the training set: {X_train.shape[0]}", f"\n Number of observations in the test set: {X_test.shape[0]}")

print(f"\n Number of observations in the training set (sampled data): {X_train_1.shape[0]}", f"\n Number of observations in the test set: {X_test.shape[0]}")

In [ ]:
X_train.head()

In [ ]:
y_train.head()

Check wehether there is any overlap between the training set and the test set

In [ ]:
assert set(X_train.index).isdisjoint(set(X_test.index))
assert set(X_train_1.index).isdisjoint(set(X_test.index))

No overlap between the training set and the test set.

Use LabelEncoder to transform categorical targets into integer targets

In [ ]:
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

In [ ]:
le_1 = LabelEncoder()
y_train_encoded_1 = le_1.fit_transform(y_train_1)

Training target

In [ ]:
y_train_encoded

In [ ]:
y_train_encoded_1

Test target

In [ ]:
y_test_encoded

Classes as integers

In [ ]:
np.unique(y_train_encoded)

In [ ]:
np.unique(y_train_encoded_1)

### Create pipelines to fit the models. One pipeline will fit on the raw data, and the other one will fit on the data after PCA with 10 components is applied for each model considered.

In [ ]:
pipe_logistic = Pipeline(
    [("scaler", StandardScaler()),
     ("logit_classifier", LogisticRegression(
         solver= "lbfgs", 
         max_iter=2000                 # increase to avoid convergence warning
     ))
    ]
)

pipe_logistic_pca = Pipeline(
    [("scaler", StandardScaler()),
     ("pca", PCA(n_components=10)),
     ("logistic_classifier", LogisticRegression(
         solver = "lbfgs", #saga is fater for larger datasets
         max_iter=2000                 # increase to avoid convergence warning
     ))
    ]
)

# Define LDA pipelines: one raw (full data), one with PCA10
lda_raw = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LinearDiscriminantAnalysis())
])

lda_pca10 = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=10, random_state=42)),
    ("clf", LinearDiscriminantAnalysis())
])


# Define QDA pipelines: one raw (full data), one with PCA10
qda_raw = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", QuadraticDiscriminantAnalysis(reg_param=0.01))
])

qda_pca10 = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=10, random_state=42)),
    ("clf", QuadraticDiscriminantAnalysis(reg_param=0.01))
])

# Define Random Forest pipelines: one raw (full data), one with PCA10
rf_raw = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", RandomForestClassifier(
        criterion='gini',
        n_estimators=100,
        max_depth=None,
        min_samples_split=2,
        max_features='sqrt'))
])

rf_pca10 = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=10, random_state=42)),
    ("clf", RandomForestClassifier(
        criterion='gini',
        n_estimators=100,
        max_depth=None,
        min_samples_split=2,
        max_features='sqrt'))
])

# Define GBDT pipelines: one raw (full data), one with PCA10
gbdt_raw = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.15,
        max_depth=2,
        subsample=0.7,
        max_features="sqrt",
        random_state=42))
])

gbdt_pca10 = Pipeline([
    ("scaler", StandardScaler()),
     ("pca", PCA(n_components=10, random_state=42)),
    ("clf", GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.15,
        max_depth=2,
        subsample=0.7,
        max_features="sqrt",
        random_state=42))
])

# ==============
# KNN pipelines
# ==============

knn_raw = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", KNeighborsClassifier(
        n_neighbors=7,
        weights="distance"
    ))
])

knn_pca10 = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=10, random_state=42)),
    ("clf", KNeighborsClassifier(
        n_neighbors=7,
        weights="distance"
    ))
])

# ==============
# SVM pipelines
# ==============

svm_raw = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", SVC(
        kernel="rbf",
        C=20,
        gamma="scale",
        probability=True
    ))
])

svm_pca10 = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=10, random_state=42)),
    ("clf", SVC(
        kernel="rbf",
        C=20,
        gamma="scale",
        probability=True
    ))
])


### Cross Validation performance on the training set

Create a list of metrics that we will use for evaluation

In [ ]:
scoring_list = {"Accuracy": "accuracy", "Bal. Acc": "balanced_accuracy", "AUC": "roc_auc_ovr", "F1":"f1_macro"}
#Macro means that we are taking the average over the number of classes - sensitive to class imbalance.
#ovr means that we treat one class as positive, the rest as negative for all the classes

Corss validation performance of logit

In [ ]:
logit_cv = evaluate_cv(pipe_logistic, X_train, y_train_encoded, scoring_list=scoring_list, model_name="Logistic") # Approx 8 mins

In [ ]:
logit_cv

Cross validation performance of LDA

In [ ]:
lda_cv_raw = evaluate_cv(lda_raw, X_train, y_train, scoring_list=scoring_list, model_name="LDA Raw")

In [ ]:
lda_cv_raw

Cross validation performance of QDA

In [ ]:
qda_cv_raw = evaluate_cv(qda_raw, X_train, y_train, scoring_list=scoring_list, model_name="QDA Raw")

In [ ]:
qda_cv_raw

Cross validation performance of Random Forest 

In [ ]:
rf_cv_raw = evaluate_cv(rf_raw, X_train, y_train, scoring_list=scoring_list, model_name="RF Raw") # Approx 10 mins

In [ ]:
rf_cv_raw

Cross validation performance of GBDT

In [ ]:
gbdt_cv_raw = evaluate_cv(gbdt_raw, X_train, y_train, scoring_list=scoring_list, model_name="GBDT Raw") # Approx 11 mins

In [ ]:
gbdt_cv_raw

In [ ]:
knn_cv_raw=evaluate_cv(knn_raw, X_train_1, y_train_1, scoring_list=scoring_list, model_name="KNN RAW")

In [ ]:
knn_cv_raw

In [ ]:
svm_cv_raw=evaluate_cv(svm_raw, X_train_1, y_train_1, scoring_list=scoring_list, model_name="SVM RAW") # Approx 2 mins

In [ ]:
svm_cv_raw

## Task 1.3 Model Evaluation on the test set and performance visualization

In this part of the project, we will report the performances of each classifier on the test set, which is composed of 20% of the observations. We will also visualize the confusion matrix of each model, as it provides an intuitive visualization of the performance of each classifier.

Create a list of models to test

In [ ]:
#Create list of models and model names
models_list_raw = [("Logistic", pipe_logistic), 
                   ("LDA", lda_raw),
                   ("QDA", qda_raw),
                   ("RF", rf_raw),
                   ("GBDT", gbdt_raw)]

models_list_pca = [("Logistic_PCA10", pipe_logistic_pca),
                   ("LDA_PCA10", lda_pca10),
                   ("QDA_PCA10", qda_pca10),
                   ("RF_PCA10", rf_pca10),
                   ("GBDT_PCA10", gbdt_pca10)]

#--------------------------------------------->>
# USING THE SAMPLED DATA
#--------------------------------------------->>

models_list_raw_1 = [
    ("KNN", knn_raw),
    ("SVM", svm_raw)
]

models_list_pca_1 = [
    ("KNN_PCA10", knn_pca10),
    ("SVM_PCA10", svm_pca10)
]


Find the performance and predictions of each model on the test set.

In [ ]:
# Find the predictions and the performance of each model, both the raw and the PCA
# Approx 11 mins
df_performances_raw, df_predictions_raw = multiple_predict_evaluate(models_list_raw, X_train, X_test, y_train_encoded, y_test_encoded)
df_performances_pca, df_predictions_pca = multiple_predict_evaluate(models_list_pca, X_train, X_test, y_train_encoded, y_test_encoded)

Performances of each model on the test set.

In [ ]:
# SEPARATELY FOR KNN AND SVM USING THE SUBSAMPLED DATA

# Find the predictions and the performance of each model, both the raw and the PCA
# Though we has to train on subset, we can stull test on original 20% test set to ensure comparability acorss all models
df_performances_raw_1, df_predictions_raw_1 = multiple_predict_evaluate(models_list_raw_1, X_train_1, X_test, y_train_encoded_1, y_test_encoded)
df_performances_pca_1, df_predictions_pca_1 = multiple_predict_evaluate(models_list_pca_1, X_train_1, X_test, y_train_encoded_1, y_test_encoded)

In [ ]:
df_performances_raw = df_performances_raw.set_index("model")
df_performances_raw

In [ ]:
# SEPARATELY FOR KNN AND SVM USING THE SUBSAMPLED DATA
df_performances_raw_1= df_performances_raw_1.set_index("model")
df_performances_raw_1

In [ ]:
df_performances_pca = df_performances_pca.set_index("model")
df_performances_pca

In [ ]:
# SEPARATELY FOR KNN AND SVM USING THE SUBSAMPLED DATA
df_performances_pca_1 = df_performances_pca_1.set_index("model")
df_performances_pca_1

Combined test performance for easy reading

In [ ]:
df_all_raw = pd.concat([df_performances_raw, df_performances_raw_1], axis=0)
df_all_raw

In [ ]:
df_all_pca = pd.concat([df_performances_pca, df_performances_pca_1], axis=0)
df_all_pca

Predictions of each model on the test set.

In [ ]:
df_predictions_raw

In [ ]:
df_predictions_raw_1

In [ ]:
df_predictions_pca

In [ ]:
df_predictions_pca_1

Predictions mapped to original land_types.

In [ ]:
df_pred_original_raw = df_predictions_raw.apply(le.inverse_transform)
df_pred_original_raw

In [ ]:
# SEPARATELY FOR KNN AND SVM USING THE SUBSAMPLED DATA
df_pred_original_raw_1 = df_predictions_raw_1.apply(le_1.inverse_transform)
df_pred_original_raw_1

In [ ]:
df_pred_original_pca = df_predictions_pca.apply(le.inverse_transform)
df_pred_original_pca

In [ ]:
# SEPARATELY FOR KNN AND SVM USING THE SUBSAMPLED DATA
df_pred_original_pca_1 = df_predictions_pca_1.apply(le_1.inverse_transform)
df_pred_original_pca_1

### Confusion matrices

Confusion matrix for Logitstic.

In [ ]:
#Raw logistic confusion matrix
plot_confusion_matrix(y_test_encoded, df_predictions_raw["Logistic"], le.classes_, "true", "Logistic")

In [ ]:
#PCA logistic confusion matrix
plot_confusion_matrix(y_test_encoded, df_predictions_pca["Logistic_PCA10"], le.classes_, "true", "Logistic PCA 10")

Confusion matrix for LDA.

In [ ]:
plot_confusion_matrix(y_test_encoded, df_predictions_raw["LDA"], le.classes_, "true", "LDA")

In [ ]:
plot_confusion_matrix(y_test_encoded, df_predictions_pca["LDA_PCA10"], le.classes_, "true", "LDA PCA10")

Confusion matrix for QDA.

In [ ]:
plot_confusion_matrix(y_test_encoded, df_predictions_raw["QDA"], le.classes_, "true", "QDA")

In [ ]:
plot_confusion_matrix(y_test_encoded, df_predictions_pca["QDA_PCA10"], le.classes_, "true", "QDA PCA 10")

Confusion Matrix for Random Forest.

In [ ]:
#Raw Random Forest confusion matrix
plot_confusion_matrix(y_test_encoded, df_predictions_raw["RF"], le.classes_, "true", "Random Forest")

In [ ]:
#PCA Random Forest confusion matrix
plot_confusion_matrix(y_test_encoded, df_predictions_pca["RF_PCA10"], le.classes_, "true", "Random Forest PCA 10")

Confusion Matrix for GBDT.

In [ ]:
#Raw GBDT confusion matrix
plot_confusion_matrix(y_test_encoded, df_predictions_raw["GBDT"], le.classes_, "true", "GBDT")

In [ ]:
#PCA GBDT confusion matrix
plot_confusion_matrix(y_test_encoded, df_predictions_pca["GBDT_PCA10"], le.classes_, "true", "GBDT PCA 10")

In [ ]:
#KNN
plot_confusion_matrix(y_test_encoded, df_predictions_raw_1["KNN"], le_1.classes_, "true", "KNN")

In [ ]:
#KNN PCA10
plot_confusion_matrix(y_test_encoded, df_predictions_pca_1["KNN_PCA10"], le_1.classes_, "true", "KNN PCA 10")

In [ ]:
#SVM 
plot_confusion_matrix(y_test_encoded, df_predictions_raw_1["SVM"], le_1.classes_, "true", "SVM")

In [ ]:
# SVM PCA10
plot_confusion_matrix(y_test_encoded, df_predictions_pca_1["SVM_PCA10"], le_1.classes_, "true", "SVM PCA 10")

#### Some observations worth noting:

We can see that the classifiers have very high performance on this dataset, despite its large size. One possible reasons is due to the fact that in image recognition the signal to noise ratio is very high, as compared to, for example, financial time series, where the signal to noise ratio is low. Also, LDA and QDA have modest performances on this dataset, compared to the other models. This suggests that the generative approach to classifier is not necessarily suitable for this dataset, especially assuming that the features follow a conditional Multivariate Normal Distribution. QDA does improved the modelling a bit from LDA but not enough to boost our prediction confidence.

We will create the function my_predict that reads a test file from the current directory, and predicts the land_types for each observation.

In [ ]:
def mypredict():

    #Define the model
    model_classifier = Pipeline(
    [("scaler", StandardScaler()),
     ("pca", PCA(n_components=10)),
     ("logistic_classifier", LogisticRegression(
         solver = "lbfgs", #saga is fater for larger datasets
         max_iter=2000                 # increase to avoid convergence warning
     ))
    ]
    )

    #Use the whole training data to fit the model
    X = data1.drop(columns=["land_type","rgb_hex", "overlay_hex"])
    y = data1["land_type"]
    lbl = LabelEncoder()
    y_encoded = lbl.fit_transform(y)
    model_classifier.fit(X, y)

    #Read the test data
    test_data1  = pd.read_csv("test.csv.gz")
    X_test_data1 = test_data1.drop(columns = ["land_type", "rgb_hex", "overlay_hex"])
    y_test_data1 = test_data1["land_type"]
    y_test_data1_encoded = lbl.transform(y_test_data1)

    #Find the predictions
    test_data1_predictions = model_classifier.predict(X_test_data1)

    #Convert into dataframe
    df_data1_predictions = pd.DataFrame({"land_type": test_data1_predictions})

    #Map back to original classes
    df_data1_pred_original = df_data1_predictions.apply(lbl.inverse_transform)





## Task 1.4 Classifying glacier ice

In this part of the project, we will solve a binary classification problem. Specifically, we will frame glacier detection as a binary task with glacier ice as the positive class, and the other land types as negative. Based on the performances on the multiclass classification problem, we will choose three models: Logistic Regression fitted on the first 10 principal components, Support Vector Machines using the raw version, and Random Forest classifier on the raw data.